In [16]:
!pip install -q TTS langchain ollama torch langchain_community  sentence_transformers spacy langchain_qdrant numpy spacy gradio pypdf2 langchain-huggingface langchain_groq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.22.0 which is incompatible.
cudf-cu12 24.4.1 requires numpy<2.0a0,>=1.23, but you have numpy 1.22.0 which is incompatible.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.1 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.22.0 which is incompatible.
plotnine 0.12.4 requires numpy>=1.23.0, but you have numpy 1.22.0 which is incompatible.
pywavelets 1.6.0 requires numpy<3,>=1.22.4, but you have numpy 1.22.0 which is incompatible.
rmm-cu12 24.4.0 requires numpy<2.0a0,>=1.23, but you have numpy 1.22.0 which is incompatible.
statsmodels 0.14.2 requires numpy>=1.22.3, but yo

In [12]:
!git clone https://github.com/Rudrabha/Wav2Lip.git

fatal: destination path 'Wav2Lip' already exists and is not an empty directory.


In [3]:
pip install -r /content/Wav2Lip/requirements.txt

  Using cached librosa-0.7.0.tar.gz (1.6 MB)
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.17.1.zip (6.5 MB)
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement opencv-python==4.1.0.25 (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.39, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.61, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.42, 4.4.0.44, 4.4.0.46, 4.5.1.48, 4.5.3.56, 4.5.4.58, 4.5.4.60, 4.5.5.62, 4.5.5.64, 4.6.0.66, 4.7.0.68, 4.7.0.72, 4.8.0.74, 4.8.0.76, 4.8.1.78, 4.9.0.80, 4.10.0.82, 4.10.0.84)
ERROR: No matching distribution found for opencv-python==4.1.0.25


In [13]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 40.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [17]:
pip install --upgrade numpy


  Using cached numpy-2.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:
      Successfully uninstalled numpy-1.22.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.22.0 requires numpy==1.22.0; python_version <= "3.10", but you have numpy 2.0.0 which is incompatible.
astropy 5.3.4 requires numpy<2,>=1.21, but you have numpy 2.0.0 which is incompatible.
cudf-cu12 24.4.1 requires numpy<2.0a0,>=1.23, but you have numpy 2.0.0 which is incompatible.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.1 which is incompatible.
cupy-cuda12x 12.2.0 requires numpy<1.27,>=1.20, but you have numpy 2.0.0 which is incompatib

In [20]:
groq_api_key = ""

In [ ]:
import gradio as gr
from PyPDF2 import PdfReader
import os
import subprocess
import uuid
import spacy
from langchain.text_splitter import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from qdrant_client.http import models
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
import numpy as np
from langchain_groq import ChatGroq

  # replace with actual method to fetch the key

# Initialize LLM
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key=groq_api_key)

# Custom VectorStoreRetriever class
class VectorStoreRetriever:
    def __init__(self, client, collection_name, embedding_function):
        self.client = client
        self.collection_name = collection_name
        self.embedding_function = embedding_function

    def retrieve(self, query):
        embedding = self.embedding_function.embed_documents([query])[0]
        search_result = self.client.search(
            collection_name=self.collection_name,
            query_vector=embedding,
            limit=5  # Adjust as needed
        )
        return [hit.payload['content'] for hit in search_result]


try:
    # Ensure compatibility with the latest numpy
    import numpy as np
    np_version = np.__version__
    print(f"Using numpy version {np_version}")

    # Load spaCy model for NLP tasks
    nlp = spacy.load('en_core_web_sm')

    # Initialize text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=20,
        length_function=len,
        is_separator_regex=False
    )

    # Initialize embeddings model
    embedding_function = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en")

    # Determine the dimension of the embeddings
    sample_text = "This is a sample text to determine the embedding dimension."
    sample_embedding = embedding_function.embed_documents([sample_text])[0]
    embedding_dimension = len(sample_embedding)

    # Initialize Qdrant client with in-memory storage
    client = QdrantClient(":memory:")
    collection_name = "cv_sections"

    # Check and create collection if it doesn't exist
    if not client.collection_exists(collection_name):
        client.recreate_collection(
            collection_name=collection_name,
            vectors_config=models.VectorParams(
                size=embedding_dimension,
                distance=models.Distance.COSINE,
            ),
        )

    # Function to extract and structure CV information
    def extract_cv_info(cv_text):
        doc = nlp(cv_text)

        sections = {
            "skills": [],
            "experience": [],
            "education": []
        }

        current_section = None
        for token in doc:
            if token.text.lower() in ["skills", "experience", "education"]:
                current_section = token.text.lower()
            elif current_section:
                sections[current_section].append(token.text)

        for key in sections:
            sections[key] = " ".join(sections[key])

        return sections

    # Function to chunk text
    def chunk_text(text):
        return text_splitter.split_text(text)

    # Function to generate embeddings
    def generate_embeddings(text_chunks):
        return embedding_function.embed_documents(text_chunks)

    # Function to index data in Qdrant
    def index_data_in_qdrant(data, collection_name):
        points = []
        for section, content in data.items():
            chunks = chunk_text(content)
            embeddings = generate_embeddings(chunks)
            for i, embedding in enumerate(embeddings):
                points.append(models.PointStruct(
                    id=str(uuid.uuid4()),  # Generate a valid UUID for each point
                    vector=embedding,
                    payload={"section": section, "content": chunks[i]}
                ))

        # Upload points to Qdrant
        client.upsert(
            collection_name=collection_name,
            points=points
        )

    # Function to apply payload filtering and retrieve data
    def retrieve_data_with_filter(collection_name, filter_conditions):
        query_filter = models.Filter(
            must=[models.FieldCondition(
                key=key,
                match=models.MatchValue(value=value)
            ) for key, value in filter_conditions.items()]
        )

        # Create a dummy vector for filter-only search and ensure it's a float
        query_vector = np.zeros(embedding_dimension, dtype=np.float32)

        search_result = client.search(
            collection_name=collection_name,
            query_vector=query_vector.tolist(),  # Convert to list for Qdrant
            limit=10,
            query_filter=query_filter
        )

        return [hit.payload for hit in search_result]

    # Function to extract text from PDF
    def extract_text_from_pdf(pdf_file):
        text = ""
        pdf_reader = PdfReader(pdf_file)
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text

    # Function to convert CV to structured docs and store in Qdrant
    def convert_cv_to_structured_docs(cv_text, collection_name, filter_conditions):
        structured_data = extract_cv_info(cv_text)
        index_data_in_qdrant(structured_data, collection_name)
        return retrieve_data_with_filter(collection_name, filter_conditions)

    # Define the ChatPromptTemplate for user interaction
    template = """Answer the following question from the context
    context = {context}
    question = {question}
    """
    prompt = PromptTemplate(input_variables=["context", "question"], template=template)

    # Function to get context for query
    def get_context(query, retriever):
        return retriever.retrieve(query)

    # Function to respond to query
 # Function to respond to query
    def respond_to_query(query, retriever):
      context = get_context(query, retriever)
      response = llm.invoke(prompt.format(question=query, context=" ".join(context)))

    # Check if response is an instance of AIMessage
      if isinstance(response, AIMessage):
        # Access the content from AIMessage
          if isinstance(response.content, str):
              return response.content.strip()
          elif isinstance(response.content, list):
            # Handle list of contents, if needed
              return response.content[0].strip()  # Assuming first item in list is relevant
          else:
              raise ValueError("Unexpected content format in AIMessage")
      else:
          raise ValueError("Unexpected response format from LLM")


    # Function to convert text to speech
    def text_to_speech(text):
        command = [
            'tts',
            '--text', text,
            '--model_name', 'tts_models/multilingual/multi-dataset/xtts_v2',
            '--vocoder_name', 'vocoder_models/universal/libri-tts/wavegrad',
            '--out_path', 'output.wav',
            '--speaker_idx', 'Brenda Stern',
            '--language_idx', 'en'
        ]

        try:
            result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            print("TTS generation successful, output saved to 'output.wav'")
            with open("output.wav", "rb") as audio_file:
                return audio_file.read()
        except subprocess.CalledProcessError as e:
            print(f"An error occurred: {e.stderr.decode()}")
            return None

    # Function to create avatar video using Wav2Lip
    def create_avatar_video(audio_file_path):
        wav2lip_dir = '/content/Wav2Lip'
        os.chdir(wav2lip_dir)

        command = [
            'python', 'inference.py',
            '--checkpoint_path', 'checkpoints/wav2lip.pth',
            '--face', 'face.mp4',
            '--audio', audio_file_path
        ]

        try:
            result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            print("Avatar video generation successful")
            video_path = 'results/result_voice.mp4'
            with open(video_path, "rb") as video_file:
                return video_file.read()
        except subprocess.CalledProcessError as e:
            print(f"An error occurred: {e.stderr.decode()}")
            return None

    # Initialize the retriever
    retriever = VectorStoreRetriever(client=client, collection_name=collection_name, embedding_function=embedding_function)

    # Gradio interface
    def chatbot(cv_file, query):
        cv_text = extract_text_from_pdf(cv_file)

        filter_conditions = {}  # Define appropriate filter conditions based on requirements
        structured_docs = convert_cv_to_structured_docs(cv_text, collection_name, filter_conditions)

        response = respond_to_query(query, retriever)

        audio_data = text_to_speech(response)

        video_file_path = create_avatar_video("output.wav")

        return response, audio_data, video_file_path

    # Gradio app setup
    inputs = [
        gr.File(label="Upload CV (PDF)"),
        gr.Textbox(label="Enter your query")
    ]

    outputs = [
        gr.Textbox(label="Response"),
        gr.Audio(label="Audio Response"),
        gr.Video(label="Avatar Video Response")
    ]

    gr.Interface(fn=chatbot, inputs=inputs, outputs=outputs, title="CV Chatbot").launch(debug=True)

except Exception as e:
    print(f"An error occurred: {str(e)}")







Using numpy version 1.22.0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
<ipython-input-25-21bc01d9673f>:68: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://4489a0d82ff143df23.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 532, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1928, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1514, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    r